In [ ]:
#Préparation des outils:
import pandas as pd
from IPython.display import display
import numpy as np
from PIL import Image

<h2>pour l'analyse de nos Convnets on procède d'abord avec des modèles simples et petits:</h2>
<h5>0- 3 couches:</h5>
    -filters entre 32 et 128 
    -kernel  entre 2 et 5
    -avec/sans MaxPooling
    -avec/sans Régularisation
    -avec/sans Dropout
    
<h5>3 à 6 couches:</h5>
    -filters entre 32 et 256 
    -kernel  entre 2 et 5
    -avec/sans MaxPooling
    -avec/sans Régularisation
    -avec/sans Dropout
    
<h5>6 à 8 couches:</h5>
    -filters entre 32 et 256 
    -kernel  entre 2 et 5
    -avec/sans MaxPooling
    -avec/sans Régularisation
    -avec/sans Dropout
    
<h5>8 à 12 couches:</h5>
    -filters entre 32 et 256 
    -kernel  entre 2 et 5
    -avec/sans MaxPooling
    -avec/sans Régularisation
    -avec/sans Dropout
    
<h5>12 à 15 couches:</h5>
    -filters entre 32 et 64 
    -kernel  entre 2 et 5
    -avec/sans MaxPooling
    -avec/sans Régularisation
    -avec/sans Dropout

<h5>tout en combinant avec des Randoms sur le type de fonction d'activation, les kernels,
et valeurs des l1, l2 et dropout on évolue les résultats 


<h3>J'utilise un code de specification pour les régularisations dans l'ordre suivant:
 Use Maxpool | Use Dropout | Use regul conv | Use regul output ]
 cela m'aide à bien mener mes tests
    <h5>--------------------------------------------------------------------------------------------------------------------
    -----------------------------------------------------------------------------------------</h5>
 <h4>Donc par exemple : ONOO : signifie que j'utilise un maxPool et une régularisation sur les CONV et pas sur la couche d'output ni un dropout
 

In [118]:
#statistiques sur ce modèle:
def print_statistics(history_df, history_df_desc):
    # Calculating statistics on all models
    if(not history_df.empty):
        sorted_train_accuracy = np.sort(history_df.train_accuracy)
        sorted_val_accuracy = np.sort(history_df.val_accuracy)

        max_val_accuracy = sorted_val_accuracy[-1]
#         second_max_val_accuracy = sorted_val_accuracy[-2]
        min_val_accuracy = history_df.val_accuracy.min()
        mean_val_accuracy = history_df.val_accuracy.mean()
        median_val_accuracy = history_df.val_accuracy.median()

        max_train_accuracy = sorted_train_accuracy[-1]
#         second_max_train_accuracy = sorted_train_accuracy[-2]
        min_train_accuracy = history_df.train_accuracy.min()
        mean_train_accuracy = history_df.train_accuracy.mean()
        median_train_accuracy = history_df.train_accuracy.median()

        # Printing models statistics
        print()
        print("########### Models {} ###########".format(history_df_desc))
        print(history_df.shape)
        print("Pire résultat sur le train : ", min_train_accuracy)
        print("Pire Résultat sur la val : ", min_val_accuracy)

        print("Meilleur résultat sur le train : ", max_train_accuracy)
        print("Meilleur résultat sur la val : ", max_val_accuracy)

        print("Moyenne des résultats sur le train : ", mean_train_accuracy)
        print("Moyenne des résultats sur la val : ", mean_val_accuracy)

        print("Mediane des résultat sur le train : ", median_train_accuracy)
        print("Mediane des résultat sur la val : ", median_val_accuracy)

def filter_on_median_val_accuracy(results_dataframes_list):
    filtered_results_dataframes = []
    
    for i in range(len(results_dataframes_list)):

        # Calculating statistics on all models
        history_df = results_dataframes_list[i]
        median_val_accuracy = history_df.val_accuracy.median()
        mean_val_accuracy = history_df.val_accuracy.mean()
        # Filtering models to get the best ones (those whose val accuracy is higher then the median)
        
        # Si la médiane est inférieure à la moyenne on enlève cette archi, car la majorité des modèles on une mauvaise
        # accuracy, et la moyenne est gonflée par seulement une petite partie des modèles. Ce qui indique que que l'archi
        # choisie n'est pas interessante
#         if(mean_val_accuracy < median_val_accuracy):
        filtered_history = history_df[history_df.val_accuracy > median_val_accuracy]
#         else:
#         filtered_history = pd.DataFrame(columns=history_df.columns)
        filtered_results_dataframes.append(filtered_history)
        
    
    return filtered_results_dataframes


def filter_on_mean_train_accuracy(results_dataframes_list):
    filtered_results_dataframes = []
    
    for i in range(len(results_dataframes_list)):

        # Calculating statistics on all models
        history_df = results_dataframes_list[i]

#         median_train_accuracy = history_df.train_accuracy.median()
        mean_train_accuracy = history_df.train_accuracy.mean()
        # Filtering models to get the best ones (those whose val accuracy is higher then the median)
        
        # Si la médiane est inférieure à la moyenne on enlève cette archi, car la majorité des modèles on une mauvaise
        # accuracy, et la moyenne est gonflée par seulement une petite partie des modèles. Ce qui indique que que l'archi
        # choisie n'est pas interessante
        filtered_history = history_df[history_df.train_accuracy > mean_train_accuracy]
        filtered_results_dataframes.append(filtered_history)
        
    
    return filtered_results_dataframes

def concat_list_of_dataframes(list_df):
    return pd.concat(list_df)

def best_models_print(filtered_results,nbr_couches_min,nbr_couches_max):
    filtered_full_tmp=[]
    for i in filtered_results:
        filtered_full_tmp.append(pd.concat(i))
    filtered_full=concat_list_of_dataframes(filtered_full_tmp)
    print_statistics(filtered_full, 'best_CNN_{}_{}'.format(nbr_couches_min,nbr_couches_max))
    return filtered_full
def print_models_filtered(dataframe,dataframe_des):
    dt=filter_on_median_val_accuracy(dataframe)
    for i in range(len(dt)):
        history = dt[i]
        history_desc = dataframe_des[i]
        print_statistics(history, history_desc)
    return dt

def print_models(dt,dataframe_des):
    for i in range(len(dt)):
        history = dt[i]
        history_desc = dataframe_des[i]
        print_statistics(history, history_desc)

<h2>Importation des résultats des modèles à 3 couches

In [18]:
#importation des données CSV
dataframe_3_3 = []
dataframe_3_3_desc = []
root_csv_folder = "..\\historique_tests\\"
#1 à 3 couches:
print("-----------------------------modèle CNN simple---------------------------------------\n "
      "-----------------------------MaxPooling: Non, Dropout: Non,Régul:Non--------------------")
#on analyse par rapport au nobmre d'épochs: 30
df = pd.read_csv("{}tested_cnn_1_3_NNNN_32_64_history.csv".format(root_csv_folder), sep=";")
dataframe_3_3.append(df)
dataframe_3_3_desc.append("CNN_3_3_NNNN")


-----------------------------modèle CNN simple---------------------------------------
 -----------------------------MaxPooling: Non, Dropout: Non,Régul:Non--------------------


In [19]:
#utilisation des régularisations,
dataframe_3_3_reg=[]
dataframe_3_3_reg_desc=[]

df = pd.read_csv("{}tested_cnn_1_3_NNON_32_64_history.csv".format(root_csv_folder), sep=";")
dataframe_3_3_reg.append(df)
dataframe_3_3_reg_desc.append("CNN_3_3_R")


#utilisation du Dropout+regul
#résultats avec des filtres 32 à 128:
dataframe_3_3_reg_drop=[]
dataframe_3_3_reg_drop_desc=[]
 
df = pd.read_csv("{}tested_cnn_1_3_OOOO_32_128_history.csv".format(root_csv_folder), sep=";")
dataframe_3_3_reg_drop.append(df)
dataframe_3_3_reg_drop_desc.append("CNN_3_3_R_D")


<h2> Importation des résultats des covnets de 3 à 6  

In [20]:
dataframe_3_6=[]
dataframe_3_6_desc=[]
print("-----------------------------MaxPooling: NON, Dropout: Non,Régul:NON--------------------")
for i in [5,6]:
    df = pd.read_csv("{}tested_cnn_{}_{}_NNNN_32128_history.csv".format(root_csv_folder,i,i), sep=";")
    dataframe_3_6.append(df)
    dataframe_3_6_desc.append("CNN_3_6_{}".format(i))
print("--------------------- nb fichier importés:",len(dataframe_3_6))


-----------------------------MaxPooling: NON, Dropout: Non,Régul:NON--------------------
--------------------- nb fichier importés: 2


In [21]:
dataframe_3_6_reg=[]
dataframe_3_6_reg_desc=[]

#importation des données avec régularisation:
print("-----------------------------MaxPooling: OUI, Dropout: Non,Régul:OUI--------------------")

#4 couches
df = pd.read_csv("{}tested_cnn_4_4_ONNN_32_64_history.csv".format(root_csv_folder), sep=";")
dataframe_3_6_reg.append(df)
dataframe_3_6_reg_desc.append("CNN_3_6_R_4")

#5 couches:

df = pd.read_csv("{}tested_cnn_5_5_ONON_64_256_history.csv".format(root_csv_folder), sep=";")
dataframe_3_6_reg.append(df)
dataframe_3_6_reg_desc.append("CNN_3_6_R_5")
#6 couches:

df = pd.read_csv("{}tested_cnn_6_6_ONON_32128_history.csv".format(root_csv_folder), sep=";")
dataframe_3_6_reg.append(df)
dataframe_3_6_reg_desc.append("CNN_3_6_R_6")

print("--------------------- nb fichier importés:",len(dataframe_3_6_reg_desc))

-----------------------------MaxPooling: OUI, Dropout: Non,Régul:OUI--------------------
--------------------- nb fichier importés: 3


In [22]:
#données avec régularisation+dropout:
dataframe_3_6_reg_drop=[]
dataframe_3_6_reg_drop_desc=[]
print("-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------")
#5 couches:

df = pd.read_csv("{}tested_cnn_5_5_OOON_32_128_history.csv".format(root_csv_folder), sep=";")
dataframe_3_6_reg_drop.append(df)
dataframe_3_6_reg_drop_desc.append("CNN_3_6_R_D_5")

#6 couches:

df = pd.read_csv("{}tested_cnn_6_6_OOON_32_128_history.csv".format(root_csv_folder), sep=";")
dataframe_3_6_reg_drop.append(df)
dataframe_3_6_reg_drop_desc.append("CNN_3_6_R_D_6")


print("--------------------- nb fichier importés:",len(dataframe_3_6_reg_drop_desc))

-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------
--------------------- nb fichier importés: 2


<h2> Importation des données des modèles  6 à 8 couches


In [137]:
dataframe_6_8=[]
dataframe_6_8_desc=[]
print("-----------------------------MaxPooling: NON, Dropout: NON ,Régul:NON--------------------")
df = pd.read_csv("{}tested_cnn_8_8_NNNN_32128_history.csv".format(root_csv_folder), sep=";")
dataframe_6_8.append(df)
dataframe_6_8_desc.append("CNN_6_8_8")
print("--------------------- nb fichier importés:",len(dataframe_6_8))

-----------------------------MaxPooling: NON, Dropout: NON ,Régul:NON--------------------
--------------------- nb fichier importés: 1


In [131]:
dataframe_6_8_reg=[]
dataframe_6_8_reg_desc=[]
#with regularisation:
print("-----------------------------MaxPooling: OUI, Dropout: NON ,Régul:OUI--------------------")
#7 couches:
df = pd.read_csv("{}tested_cnn_7_7_ONON_32128_history.csv".format(root_csv_folder), sep=";")
dataframe_6_8_reg.append(df)
dataframe_6_8_reg_desc.append("CNN_6_8_R_7")
#8 couches
df = pd.read_csv("{}tested_cnn_8_8_NNON_32_64_history.csv".format(root_csv_folder), sep=";")
dataframe_6_8_reg.append(df)
dataframe_6_8_reg_desc.append("CNN_6_8_R_8")
print("--------------------- nb fichier importés:",len(dataframe_6_8_reg))

-----------------------------MaxPooling: OUI, Dropout: NON ,Régul:OUI--------------------
--------------------- nb fichier importés: 2


In [133]:
dataframe_6_8_reg_drop=[]
dataframe_6_8_reg_drop_desc=[]
#with regularisation:
print("-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------")
#with dropout:
#7 couches:    
df = pd.read_csv("{}tested_cnn_7_7_OOOO_32_256_history.csv".format(root_csv_folder), sep=";")
dataframe_6_8_reg_drop.append(df)
dataframe_6_8_reg_drop_desc.append("CNN_6_8_R_D_7")
#7 couches:    
df = pd.read_csv("{}tested_cnn_8_8_OOOO_32_128_history.csv".format(root_csv_folder), sep=";")
dataframe_6_8_reg_drop.append(df)
dataframe_6_8_reg_drop_desc.append("CNN_6_8_R_D_8")
print("--------------------- nb fichier importés:",len(dataframe_6_8_reg_drop))


-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------
--------------------- nb fichier importés: 2


<h2> Importation des données des modèle de 8 à 12 couches


In [99]:
dataframe_8_12=[]
dataframe_8_12_desc=[]
print("-----------------------------MaxPooling: NON, Dropout: NON ,Régul:NON--------------------")
for i in [8,10]:
    df = pd.read_csv("{}tested_cnn_{}_{}_NNNN_32128_history.csv".format(root_csv_folder,i,i), sep=";")
    dataframe_8_12.append(df)
    dataframe_8_12_desc.append("CNN_8_12_{}".format(i))
print("--------------------- nb fichier importés:",len(dataframe_8_12))

-----------------------------MaxPooling: NON, Dropout: NON ,Régul:NON--------------------
--------------------- nb fichier importés: 2


In [103]:
dataframe_8_12_reg=[]
dataframe_8_12_reg_desc=[]
#with regularisation:
print("-----------------------------MaxPooling: OUI, Dropout: NON ,Régul:OUI--------------------")

#8 couches
df = pd.read_csv("{}tested_cnn_8_8_NNON_32_64_history.csv".format(root_csv_folder), sep=";")
dataframe_8_12_reg.append(df)
dataframe_8_12_reg_desc.append("CNN_8_12_R_8")

# #10 couches
# df = pd.read_csv("{}tested_cnn_10_ONOO_32128_history.csv".format(root_csv_folder), sep=";")
# dataframe_8_12_reg.append(df)
# dataframe_8_12_reg_desc.append("CNN_8_12_R_10")

print("--------------------- nb fichier importés:",len(dataframe_8_12_reg))

-----------------------------MaxPooling: OUI, Dropout: NON ,Régul:OUI--------------------
--------------------- nb fichier importés: 1


In [104]:
dataframe_8_12_reg_drop=[]
dataframe_8_12_reg_drop_desc=[]
#with regularisation:
print("-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------")
#with dropout:
#8 couches:    
df = pd.read_csv("{}tested_cnn_8_8_OOOO_32_128_history.csv".format(root_csv_folder), sep=";")
dataframe_8_12_reg_drop.append(df)
dataframe_8_12_reg_drop_desc.append("CNN_8_12_R_D_8")
#10 couches:    
df = pd.read_csv("{}tested_cnn_1010_OOOO_32_128_history.csv".format(root_csv_folder), sep=";")
dataframe_8_12_reg_drop.append(df)
dataframe_8_12_reg_drop_desc.append("CNN_8_12_R_D_10")
print("--------------------- nb fichier importés:",len(dataframe_8_12_reg_drop))

-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------
--------------------- nb fichier importés: 2


<h2>Importation des données des modèles  de 12 à 15 couches

In [108]:
dataframe_12_15=[]
dataframe_12_15_desc=[]
print("-----------------------------MaxPooling: NON, Dropout: NON ,Régul:NON--------------------")
for i in [12]:
    df = pd.read_csv("{}tested_cnn_{}_NNNN_32128_history.csv".format(root_csv_folder,i), sep=";")
    dataframe_12_15.append(df)
    dataframe_12_15_desc.append("CNN_12_15_{}".format(i))
print("--------------------- nb fichier importés:",len(dataframe_12_15))

-----------------------------MaxPooling: NON, Dropout: NON ,Régul:NON--------------------
--------------------- nb fichier importés: 1


In [154]:
dataframe_12_15_reg=[]
dataframe_12_15_reg_desc=[]
#with regularisation:
print("-----------------------------MaxPooling: OUI, Dropout: NON ,Régul:OUI--------------------")

df = pd.read_csv("{}tested_cnn_15_ONOO_32128_history.csv".format(root_csv_folder), sep=";")
dataframe_12_15_reg.append(df)
dataframe_12_15_reg_desc.append("CNN_12_15_R_{}".format(i))

print("--------------------- nb fichier importés:",len(dataframe_12_15_reg))


-----------------------------MaxPooling: OUI, Dropout: NON ,Régul:OUI--------------------
--------------------- nb fichier importés: 1


In [155]:
dataframe_12_15_reg_drop=[]
dataframe_12_15_reg_drop_desc=[]
#with regularisation:
print("-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------")
#with dropout:
#12 couches:    
df = pd.read_csv("{}tested_cnn_12_12_OOON_32128_history.csv".format(root_csv_folder), sep=";")
dataframe_12_15_reg_drop.append(df)
dataframe_12_15_reg_drop_desc.append("CNN_12_15_R_D_12")
#15 couches:    
df = pd.read_csv("{}tested_cnn_1515_OOON_32128_history.csv".format(root_csv_folder), sep=";")
dataframe_12_15_reg_drop.append(df)
dataframe_12_15_reg_drop_desc.append("CNN_12_15_R_D_15")
print("--------------------- nb fichier importés:",len(dataframe_12_15_reg_drop))

-----------------------------MaxPooling: OUI, Dropout: OUI ,Régul:OUI--------------------
--------------------- nb fichier importés: 2


<h1>Partie de test:
<h2>TEST DU CNN à 3 couches sans régularisation :

In [34]:
print_models(dataframe_3_3,dataframe_3_3_desc)


########### Models CNN_3_3_NNNN ###########
(5, 24)
Pire résultat sur le train :  0.81542
Pire Résultat sur la val :  0.4682
Meilleur résultat sur le train :  0.9995200000000001
Meilleur résultat sur la val :  0.5709
Moyenne des résultats sur le train :  0.9190240000000001
Moyenne des résultats sur la val :  0.51976
Mediane des résultat sur le train :  0.9146
Mediane des résultat sur la val :  0.5085


 
Pour un début les résultats de la val semblent bons, 
mais on remarque que le modèle simple  représente un grand overfiting,
essayons de prendre que les meilleurs modèles
<h5>   meilleurs modèles:

In [35]:
filtered_3_3=print_models_filtered(dataframe_3_3,dataframe_3_3_desc)


########### Models CNN_3_3_NNNN ###########
(2, 24)
Pire résultat sur le train :  0.9146
Pire Résultat sur la val :  0.5585
Meilleur résultat sur le train :  0.9995200000000001
Meilleur résultat sur la val :  0.5709
Moyenne des résultats sur le train :  0.95706
Moyenne des résultats sur la val :  0.5647
Mediane des résultat sur le train :  0.95706
Mediane des résultat sur la val :  0.5647


<h5> on remarque donc que notre modèle arrive quand même à atteindre 0.57 en vac accuracy , et qu'il dépasse de peu de la moyenne, et la médiane,
cependant les valeur du train sont largement en Overfiting
raison pour laquelle on doit utiliser des techniques de régularisation l1 l2 et dropout
</h5>   

<h2> Test Du CNN à 3 couches avec la régularisation:</h2>

In [36]:
print_models(dataframe_3_3_reg,dataframe_3_3_reg_desc)


########### Models CNN_3_3_R ###########
(17, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.9958600000000001
Meilleur résultat sur la val :  0.6737
Moyenne des résultats sur le train :  0.6050070588235293
Moyenne des résultats sur la val :  0.47359999999999997
Mediane des résultat sur le train :  0.63564
Mediane des résultat sur la val :  0.5542


affichage des meilleurs modèles:

In [37]:
filtered_3_3_reg = print_models_filtered(dataframe_3_3_reg,dataframe_3_3_reg_desc)


########### Models CNN_3_3_R ###########
(8, 24)
Pire résultat sur le train :  0.57868
Pire Résultat sur la val :  0.5543
Meilleur résultat sur le train :  0.9958600000000001
Meilleur résultat sur la val :  0.6737
Moyenne des résultats sur le train :  0.75898
Moyenne des résultats sur la val :  0.6006374999999999
Mediane des résultat sur le train :  0.74793
Mediane des résultat sur la val :  0.57735


On voit que sur les 17 modèles lancés 8 sont pertinents et arrivent à 0.67 sur la val mais le modèle overfite dans la médiane et la moyenne
,essayons peut être de le rigulariser encore mieux avec un dropout:
<h2>TesT Du CNN à 3 couches avec la régularisation+Droupout:

In [38]:
print_models(dataframe_3_3_reg_drop,dataframe_3_3_reg_drop_desc)


########### Models CNN_3_3_R_D ###########
(6, 24)
Pire résultat sur le train :  0.23606
Pire Résultat sur la val :  0.2392
Meilleur résultat sur le train :  0.98006
Meilleur résultat sur la val :  0.6513
Moyenne des résultats sur le train :  0.6844866666666666
Moyenne des résultats sur la val :  0.5543833333333333
Mediane des résultat sur le train :  0.6591199999999999
Mediane des résultat sur la val :  0.61975


on remarque déja que l'overfiting a baissé par rapport à l'étape d'avant,
celà est dû à une utilisation du dropout de 0.25 jusqu'à 0.56
<h5> les meilleurs modèles

In [144]:
filtered_3_3_reg_drop=print_models_filtered(dataframe_3_3_reg_drop,dataframe_3_3_reg_drop_desc)


########### Models CNN_3_3_R_D ###########
(3, 24)
Pire résultat sur le train :  0.6471399999999999
Pire Résultat sur la val :  0.6218
Meilleur résultat sur le train :  0.9293600000000001
Meilleur résultat sur la val :  0.6513
Moyenne des résultats sur le train :  0.7492
Moyenne des résultats sur la val :  0.6358666666666667
Mediane des résultat sur le train :  0.6711
Mediane des résultat sur la val :  0.6345


On ne sélectionne que 2 modèles pertinents,mais le modèle n'est pas aussi fort , d'où on va essayer d'augmenter le nombre de couches Conv
<h2>Conclusion :
Nous allons regrouper tous ce meilleurs modèles à 3 couches  

In [145]:
best_models_print([filtered_3_3,filtered_3_3_reg,filtered_3_3_reg_drop],3,3).sort_values('val_accuracy',ascending=False)


########### Models best_CNN_3_3 ###########
(13, 24)
Pire résultat sur le train :  0.57868
Pire Résultat sur la val :  0.5543
Meilleur résultat sur le train :  0.9995200000000001
Meilleur résultat sur la val :  0.6737
Moyenne des résultats sur le train :  0.7871969230769232
Moyenne des résultats sur la val :  0.6032384615384616
Mediane des résultat sur le train :  0.8124
Mediane des résultat sur la val :  0.5801


,Conv2D_nb,Conv2D_size,Conv2D_activation,output_activation,Conv2D_padding,MaxPooling2D_use,MaxPooling2D_values,MaxPooling2D_Position,use_dropout,dropout_indexes,...,l2_value,regul_kernel_indexes,loss,optimizer,metrics,epochs,model_id,train_accuracy,val_accuracy,dossier
5,3,"(256, 3) (256, 3) (128, 3)",selu,softmax,same,True,3,1,False,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,4086724,0.90286,0.6737,20200126
2,3,"(64, 2) (64, 2) (32, 2)",softplus,softmax,same,True,2,1 2,False,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,7884044,0.81240,0.6723,20200126
1,3,"(128, 3) (64, 3) (32, 3)",relu,softmax,same,True,3,2,True,0.0,...,0.040,1 3,sparse_categorical_crossentropy,str,accuracy,35,5445085,0.67110,0.6513,20200206
4,3,"(32, 2) (64, 2) (128, 2)",selu,softmax,same,True,2,3 2 3 3 2,True,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,9819395,0.92936,0.6345,20200126
0,3,"(128, 3) (64, 3) (32, 3)",relu,softmax,same,True,3,2,True,0.0,...,0.040,1 3,sparse_categorical_crossentropy,str,accuracy,30,6479636,0.64714,0.6218,20200206
12,3,"(64, 2) (256, 2) (64, 2)",softplus,softmax,same,True,2,3 2 2 2 1,False,NaN,...,0.030,3 3,sparse_categorical_crossentropy,str,['accuracy'],30,228629,0.63564,0.6082,20200126
3,2,"(64, 5) (64, 5)",softmax,softmax,same,True,5,2,False,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,2029876,0.99586,0.5801,20200126
16,3,"(32, 2) (32, 2) (64, 2)",softmax,softmax,same,False,2,NaN,True,1.0,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,8092382,0.68346,0.5746,20200125
4,2,"(32, 2) (32, 2)",relu,softmax,same,True,2,2 1,False,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,2056139,0.82958,0.5712,20200126
2,2,"(32, 2) (32, 2)",softmax,softmax,same,False,2,NaN,False,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,6953084,0.91460,0.5709,20200126


In [48]:
best_models = filter_on_mean_train_accuracy([best_val_models])[0]
best_models = best_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_models, "Best CNN 3_3 models")


########### Models Best MLP models ###########
(7, 24)
Pire résultat sur le train :  0.8124
Pire Résultat sur la val :  0.5585
Meilleur résultat sur le train :  0.9995200000000001
Meilleur résultat sur la val :  0.6737
Moyenne des résultats sur le train :  0.9120257142857143
Moyenne des résultats sur la val :  0.6087428571428573
Mediane des résultat sur le train :  0.9146
Mediane des résultat sur la val :  0.5801


## voici donc le meilleur modèle obtenu:


![Best 3 3](./images_from_tensorBoard/1_3_best.PNG)

on pourra alors augmenter le nombre couches de conv au vu de nos résultats
<h2> tests 3 à 6 </h2>
<h2> Résultats du modèle 3 à 6 couches sans régularisation:

In [193]:
print_models(dataframe_3_6,dataframe_3_6_desc)


########### Models CNN_3_6_5 ###########
(3, 24)
Pire résultat sur le train :  0.91346
Pire Résultat sur la val :  0.5847
Meilleur résultat sur le train :  0.9807600000000001
Meilleur résultat sur la val :  0.613
Moyenne des résultats sur le train :  0.9483266666666669
Moyenne des résultats sur la val :  0.6002333333333333
Mediane des résultat sur le train :  0.95076
Mediane des résultat sur la val :  0.603

########### Models CNN_3_6_6 ###########
(2, 24)
Pire résultat sur le train :  0.8817
Pire Résultat sur la val :  0.5641
Meilleur résultat sur le train :  0.9763799999999999
Meilleur résultat sur la val :  0.5941
Moyenne des résultats sur le train :  0.92904
Moyenne des résultats sur la val :  0.5791
Mediane des résultat sur le train :  0.92904
Mediane des résultat sur la val :  0.5791


In [202]:
filtered_3_6=print_models_filtered(dataframe_3_6,dataframe_3_6_desc)


########### Models CNN_3_6_5 ###########
(1, 24)
Pire résultat sur le train :  0.9807600000000001
Pire Résultat sur la val :  0.613
Meilleur résultat sur le train :  0.9807600000000001
Meilleur résultat sur la val :  0.613
Moyenne des résultats sur le train :  0.9807600000000001
Moyenne des résultats sur la val :  0.613
Mediane des résultat sur le train :  0.9807600000000001
Mediane des résultat sur la val :  0.613

########### Models CNN_3_6_6 ###########
(1, 24)
Pire résultat sur le train :  0.8817
Pire Résultat sur la val :  0.5941
Meilleur résultat sur le train :  0.8817
Meilleur résultat sur la val :  0.5941
Moyenne des résultats sur le train :  0.8817
Moyenne des résultats sur la val :  0.5941
Mediane des résultat sur le train :  0.8817
Mediane des résultat sur la val :  0.5941


étant donnée qu'on a essayé que 2 modèles par nbr couches, on remarque clairement que les modèles overfitent, et la val 
accuracy à baissé par rapport au modèle à 3 couches, les résultats sont prochent à la moyenne et médiane,
on va tenter une régularisation pour contrôler l'overfiting 
<h5> </t> Test avec Régularisation

In [53]:
print_models(dataframe_3_6_reg,
    dataframe_3_6_reg_desc)



########### Models CNN_3_6_R_4 ###########
(9, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.8715799999999999
Meilleur résultat sur la val :  0.7037
Moyenne des résultats sur le train :  0.6542244444444445
Moyenne des résultats sur la val :  0.5792777777777778
Mediane des résultat sur le train :  0.7673
Mediane des résultat sur la val :  0.6904

########### Models CNN_3_6_R_5 ###########
(55, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.8330799999999999
Meilleur résultat sur la val :  0.7603
Moyenne des résultats sur le train :  0.7517640000000001
Moyenne des résultats sur la val :  0.700510909090909
Mediane des résultat sur le train :  0.77862
Mediane des résultat sur la val :  0.7299

########### Models CNN_3_6_R_6 ###########
(69, 24)
Pire résultat sur le train :  0.68828
Pire Résultat sur la val :  0.6579999999999999
Meilleur résultat sur le train :  0.89884
Mei

On constate clairement que les résultats ont progréssés au niveau de la val pour les modèles à 5 et à 6
couches, on remarque également que l'overfiting à baissé et s'approche de la val,
dans ce cas j'ai utilisé une (0.25<l2<0.95) et(0.05< l1 0.35)
et j'ai également changé le nombre d'épochs, celà à aidé dans la progression,
et finalement echangé l'emplacement des MaxPooling, et des L2, 
<h5> voyant les meilleurs modèles à 4 5 et 6 couches

In [54]:
filtered_3_6_reg=print_models_filtered(dataframe_3_6_reg,dataframe_3_6_reg_desc)


########### Models CNN_3_6_R_4 ###########
(4, 24)
Pire résultat sur le train :  0.7556
Pire Résultat sur la val :  0.6944
Meilleur résultat sur le train :  0.8101
Meilleur résultat sur la val :  0.7037
Moyenne des résultats sur le train :  0.7752650000000001
Moyenne des résultats sur la val :  0.6990249999999999
Mediane des résultat sur le train :  0.76768
Mediane des résultat sur la val :  0.699

########### Models CNN_3_6_R_5 ###########
(27, 24)
Pire résultat sur le train :  0.7740600000000001
Pire Résultat sur la val :  0.7309
Meilleur résultat sur le train :  0.8330799999999999
Meilleur résultat sur la val :  0.7603
Moyenne des résultats sur le train :  0.8009933333333331
Moyenne des résultats sur la val :  0.7401925925925923
Mediane des résultat sur le train :  0.80186
Mediane des résultat sur la val :  0.7375

########### Models CNN_3_6_R_6 ###########
(34, 24)
Pire résultat sur le train :  0.8025800000000001
Pire Résultat sur la val :  0.7308
Meilleur résultat sur le train : 

<h1> Ce modèle à était traité au fur et à mesure des résultats, raison pour laquelle il ya plus de 100 tests performants.</h1>
on constate donc parmi les meilleurs modèles : </n>
    <h5></t>-4 couches:</h5> val accuracy et accuracy sont proches en moyenne et en médiane,</n>
    <h5></t>-5 couches:</h5> val accuracy touche son record,et on est à 0.05 de différence en médiane
    c'est plus de 60 test d'améliorations effectuées</n>
    <h5></t>-6 couches:</h5> val accuracy touche 0.75 en revanche l'accuracy est à 0.11 ce qui est
    moins pertinent que celui de la couche5 
<h5> Test avec Régularisation + Dropout:

In [197]:
print_models(dataframe_3_6_reg_drop,dataframe_3_6_reg_drop_desc)
    


########### Models CNN_3_6_R_D_5 ###########
(9, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.9098
Meilleur résultat sur la val :  0.7293
Moyenne des résultats sur le train :  0.6729777777777778
Moyenne des résultats sur la val :  0.6170666666666667
Mediane des résultat sur le train :  0.75964
Mediane des résultat sur la val :  0.6978

########### Models CNN_3_6_R_D_6 ###########
(11, 24)
Pire résultat sur le train :  0.48678
Pire Résultat sur la val :  0.4874
Meilleur résultat sur le train :  0.9660799999999999
Meilleur résultat sur la val :  0.7464
Moyenne des résultats sur le train :  0.6912690909090909
Moyenne des résultats sur la val :  0.6420090909090909
Mediane des résultat sur le train :  0.6999
Mediane des résultat sur la val :  0.6628


On remarque ici que l'overfiting à baissé dans les deux couches du modèle, celà est dù à
une utilisation d'un dropout  0.25<d<0.55 et une augmnetation de la l1 à 50% environ,
On pourra vérifier :
<h5> les meilleurs modèles générés ici :

In [198]:
filtered_3_6_reg_drop=print_models_filtered(dataframe_3_6_reg_drop,dataframe_3_6_reg_drop_desc)


########### Models CNN_3_6_R_D_5 ###########
(4, 24)
Pire résultat sur le train :  0.75964
Pire Résultat sur la val :  0.7199
Meilleur résultat sur le train :  0.7850600000000001
Meilleur résultat sur la val :  0.7293
Moyenne des résultats sur le train :  0.7691950000000001
Moyenne des résultats sur la val :  0.723675
Mediane des résultat sur le train :  0.76604
Mediane des résultat sur la val :  0.72275

########### Models CNN_3_6_R_D_6 ###########
(5, 24)
Pire résultat sur le train :  0.70024
Pire Résultat sur la val :  0.6642
Meilleur résultat sur le train :  0.9660799999999999
Meilleur résultat sur la val :  0.7464
Moyenne des résultats sur le train :  0.784628
Moyenne des résultats sur la val :  0.69998
Mediane des résultat sur le train :  0.74616
Mediane des résultat sur la val :  0.6987


<h3> Le dropout à quand même réduit l'overfiting jusqu'à seulement 0.04 en médiane,ce qui
est très utile,
malheuresement la val avait baissée entre temps,à cause de régularisations, 
<h2> On récapitule sur l'ensmeble des meilleurs résultats

In [203]:
best_models_print([filtered_3_6,filtered_3_6_reg,filtered_3_3_reg_drop],3,6).sort_values('val_accuracy',ascending=False)


########### Models best_CNN_3_6 ###########
(70, 24)
Pire résultat sur le train :  0.6471399999999999
Pire Résultat sur la val :  0.5941
Meilleur résultat sur le train :  0.9807600000000001
Meilleur résultat sur la val :  0.7603
Moyenne des résultats sur le train :  0.8214100000000001
Moyenne des résultats sur la val :  0.7287728571428572
Mediane des résultat sur le train :  0.8177099999999999
Mediane des résultat sur la val :  0.73735


,Conv2D_nb,Conv2D_size,Conv2D_activation,output_activation,Conv2D_padding,MaxPooling2D_use,MaxPooling2D_values,MaxPooling2D_Position,use_dropout,dropout_indexes,...,l2_value,regul_kernel_indexes,loss,optimizer,metrics,epochs,model_id,train_accuracy,val_accuracy,dossier
21,5,"(64, 3) (128, 3) (64, 3) (32, 3) (32, 3)",selu,softmax,same,True,3,2 4,False,0,...,0.060,1 3,sparse_categorical_crossentropy,str,accuracy,35,871806,0.83308,0.7603,20200203
9,6,"(64, 3) (128, 3) (128, 3) (64, 3) (32, 3) (32, 3)",relu,softmax,same,True,3,4 5,False,0,...,0.040,1 4,sparse_categorical_crossentropy,str,accuracy,25,4029029,0.84328,0.7547,20200201
17,6,"(64, 3) (128, 3) (128, 3) (64, 3) (32, 3) (32, 3)",relu,softmax,same,True,3,4 5,False,0,...,0.040,1 4,sparse_categorical_crossentropy,str,accuracy,40,5524285,0.89076,0.7520,20200201
30,5,"(64, 3) (128, 3) (64, 3) (32, 3) (32, 3)",selu,softmax,same,True,3,2 4,False,0,...,0.075,1 3,sparse_categorical_crossentropy,str,accuracy,45,500504,0.82690,0.7514,20200203
16,5,"(64, 3) (128, 3) (64, 3) (32, 3) (32, 3)",selu,softmax,same,True,3,2 4,False,0,...,0.054,1 3,sparse_categorical_crossentropy,str,accuracy,35,640730,0.81538,0.7503,20200203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,3,"(128, 3) (64, 3) (32, 3)",relu,softmax,same,True,3,2,True,0,...,0.040,1 3,sparse_categorical_crossentropy,str,accuracy,35,5445085,0.67110,0.6513,20200206
4,3,"(32, 2) (64, 2) (128, 2)",selu,softmax,same,True,2,3 2 3 3 2,True,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,9819395,0.92936,0.6345,20200126
0,3,"(128, 3) (64, 3) (32, 3)",relu,softmax,same,True,3,2,True,0,...,0.040,1 3,sparse_categorical_crossentropy,str,accuracy,30,6479636,0.64714,0.6218,20200206
1,5,"(64, 3) (128, 3) (64, 3) (64, 3) (32, 3)",relu,softmax,same,False,0,NaN,False,0,...,0.000,NaN,sparse_categorical_crossentropy,str,accuracy,30,3176575,0.98076,0.6130,20200208


![Best 3_6](./images_from_tensorBoard/Best_5_5.png)

en conclusion : on essayera d'augmenter le nombre de CONV du modèle tout en gardant le même
kernel et filers,et on va tenter de choper un résultat meilleur:
<h2> Résultats du modèle 6 à 8 couches sans régularisation:


In [134]:
print_models(dataframe_6_8,dataframe_6_8_desc)
filtered_6_8= print_models_filtered(dataframe_6_8,dataframe_6_8_desc)


########### Models CNN_6_8_8 ###########
(2, 24)
Pire résultat sur le train :  0.8819
Pire Résultat sur la val :  0.5676
Meilleur résultat sur le train :  0.97088
Meilleur résultat sur la val :  0.6258
Moyenne des résultats sur le train :  0.92639
Moyenne des résultats sur la val :  0.5967
Mediane des résultat sur le train :  0.92639
Mediane des résultat sur la val :  0.5967

########### Models CNN_6_8_8 ###########
(1, 24)
Pire résultat sur le train :  0.97088
Pire Résultat sur la val :  0.6258
Meilleur résultat sur le train :  0.97088
Meilleur résultat sur la val :  0.6258
Moyenne des résultats sur le train :  0.97088
Moyenne des résultats sur la val :  0.6258
Mediane des résultat sur le train :  0.97088
Mediane des résultat sur la val :  0.6258


comme on a pas beaucoup de modèles simples lancés on constate évidement un Overfiting,
 c'est normal vu qu'on continue avec les mêmes couches de l'analyse .
<h5> Test avec Régularisation

In [74]:
print_models(dataframe_6_8_reg,dataframe_6_8_reg_desc)


########### Models CNN_6_8_R_7 ###########
(19, 24)
Pire résultat sur le train :  0.6822
Pire Résultat sur la val :  0.6528
Meilleur résultat sur le train :  0.7893
Meilleur résultat sur la val :  0.7265
Moyenne des résultats sur le train :  0.7323831578947368
Moyenne des résultats sur la val :  0.6923842105263158
Mediane des résultat sur le train :  0.72388
Mediane des résultat sur la val :  0.6957

########### Models CNN_6_8_R_8 ###########
(4, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.36302
Meilleur résultat sur la val :  0.3629
Moyenne des résultats sur le train :  0.16575499999999999
Moyenne des résultats sur la val :  0.16572499999999998
Mediane des résultat sur le train :  0.1
Mediane des résultat sur la val :  0.1


In [138]:
filtered_6_8_reg=print_models_filtered(dataframe_6_8_reg,dataframe_6_8_reg_desc)


########### Models CNN_6_8_R_7 ###########
(9, 24)
Pire résultat sur le train :  0.7332
Pire Résultat sur la val :  0.6976
Meilleur résultat sur le train :  0.7893
Meilleur résultat sur la val :  0.7265
Moyenne des résultats sur le train :  0.7573444444444445
Moyenne des résultats sur la val :  0.7108111111111112
Mediane des résultat sur le train :  0.75238
Mediane des résultat sur la val :  0.7064

########### Models CNN_6_8_R_8 ###########
(3, 24)
Pire résultat sur le train :  0.36302
Pire Résultat sur la val :  0.3629
Meilleur résultat sur le train :  0.69868
Meilleur résultat sur la val :  0.6233
Moyenne des résultats sur le train :  0.5700466666666667
Moyenne des résultats sur la val :  0.5280999999999999
Mediane des résultat sur le train :  0.64844
Mediane des résultat sur la val :  0.5981


<h5>on remarque que les resultats sont extraimenet pertinenets,
on remarque sur les meilleurs modèles une légère différence entre les moyenns et les médiane
c'est dù aux améliorations progressives ajoutées au fur des tests en L1 et L2
<h5> Test avec Régularisation + Dropout:

In [70]:
print_models(dataframe_6_8_reg_drop,dataframe_6_8_reg_drop_desc)


########### Models CNN_6_8_R_D_7 ###########
(7, 24)
Pire résultat sur le train :  0.33884000000000003
Pire Résultat sur la val :  0.3343
Meilleur résultat sur le train :  0.6240600000000001
Meilleur résultat sur la val :  0.6109
Moyenne des résultats sur le train :  0.51038
Moyenne des résultats sur la val :  0.5028142857142858
Mediane des résultat sur le train :  0.58614
Mediane des résultat sur la val :  0.5773

########### Models CNN_6_8_R_D_8 ###########
(27, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.9919600000000001
Meilleur résultat sur la val :  0.7313
Moyenne des résultats sur le train :  0.7075466666666669
Moyenne des résultats sur la val :  0.5433444444444445
Mediane des résultat sur le train :  0.86144
Mediane des résultat sur la val :  0.6567


 
<h5>les meilleurs modèles sont:

In [71]:
filtered_6_8_reg_drop=print_models_filtered(dataframe_6_8_reg_drop,dataframe_6_8_reg_drop_desc)    



########### Models CNN_6_8_R_D_7 ###########
(3, 24)
Pire résultat sur le train :  0.6040399999999999
Pire Résultat sur la val :  0.5955
Meilleur résultat sur le train :  0.6240600000000001
Meilleur résultat sur la val :  0.6109
Moyenne des résultats sur le train :  0.6147733333333333
Moyenne des résultats sur la val :  0.6013999999999999
Mediane des résultat sur le train :  0.61622
Mediane des résultat sur la val :  0.5978

########### Models CNN_6_8_R_D_8 ###########
(12, 24)
Pire résultat sur le train :  0.7261
Pire Résultat sur la val :  0.6571
Meilleur résultat sur le train :  0.9919600000000001
Meilleur résultat sur la val :  0.7313
Moyenne des résultats sur le train :  0.8975983333333333
Moyenne des résultats sur la val :  0.6923750000000001
Mediane des résultat sur le train :  0.94513
Mediane des résultat sur la val :  0.6874


On remarque ici que les résultats pour les modèles à 7 couches sont en baisse en val  mais le l'overfiting à disparu,
contrairement à celui de la couche 8 qui reste toujours en overfiting, c'est probablement du
à l2 utilisée (0.05<l2<0.15)

In [139]:
best_models_print( [filtered_6_8_reg,filtered_6_8_reg_drop],6,8).sort_values('val_accuracy',ascending=False)


########### Models best_CNN_6_8 ###########
(27, 24)
Pire résultat sur le train :  0.36302
Pire Résultat sur la val :  0.3629
Meilleur résultat sur le train :  0.9919600000000001
Meilleur résultat sur la val :  0.7313
Moyenne des résultats sur le train :  0.7830274074074076
Moyenne des résultats sur la val :  0.6701592592592591
Mediane des résultat sur le train :  0.75772
Mediane des résultat sur la val :  0.7006


,Conv2D_nb,Conv2D_size,Conv2D_activation,output_activation,Conv2D_padding,MaxPooling2D_use,MaxPooling2D_values,MaxPooling2D_Position,use_dropout,dropout_indexes,...,l2_value,regul_kernel_indexes,loss,optimizer,metrics,epochs,model_id,train_accuracy,val_accuracy,dossier
3,8,"(128, 4) (128, 4) (64, 4) (64, 4) (64, 4) (32,...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,7,sparse_categorical_crossentropy,str,accuracy,50,1760125,0.98344,0.7313,20200127
5,8,"(128, 4) (128, 4) (64, 4) (64, 4) (64, 4) (32,...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,7,sparse_categorical_crossentropy,str,accuracy,30,275590,0.95916,0.7291,20200128
4,8,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,6,sparse_categorical_crossentropy,str,accuracy,150,6817867,0.99196,0.7287,20200127
3,7,"(64, 3) (128, 3) (128, 3) (64, 3) (64, 3) (32,...",selu,softmax,same,True,3,4 5,False,0,...,0.090,1 4 6,sparse_categorical_crossentropy,str,accuracy,40,1272990,0.78930,0.7265,20200204
2,7,"(64, 3) (128, 3) (128, 3) (64, 3) (64, 3) (32,...",selu,softmax,same,True,3,4 5,False,0,...,0.090,1 4 6,sparse_categorical_crossentropy,str,accuracy,35,6628961,0.77010,0.7243,20200204
1,8,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,6,sparse_categorical_crossentropy,str,accuracy,50,1799197,0.97602,0.7199,20200127
4,7,"(64, 3) (128, 3) (128, 3) (64, 3) (64, 3) (32,...",selu,softmax,same,True,3,4 5,False,0,...,0.090,1 4 6,sparse_categorical_crossentropy,str,accuracy,45,9656955,0.77776,0.7191,20200204
0,8,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,NaN,sparse_categorical_crossentropy,str,accuracy,30,6122335,0.97594,0.7174,20200127
0,7,"(64, 3) (128, 3) (128, 3) (64, 3) (64, 3) (32,...",selu,softmax,same,True,3,4 5,False,0,...,0.090,1 4 6,sparse_categorical_crossentropy,str,accuracy,25,3315946,0.75772,0.7167,20200204
18,7,"(64, 3) (128, 3) (128, 3) (64, 3) (64, 3) (32,...",selu,softmax,same,True,3,4 5,True,0 3,...,0.098,1 4 6,sparse_categorical_crossentropy,str,accuracy,35,8982795,0.73972,0.7064,20200208


<h5> Dans l'ensemble les résultats semblent pertinent pour ce normbre de couches, 
essayons de voir si on peu faire mieux ai
 <h2> Résultat test 8 à 12

In [77]:
print_models(dataframe_8_12,dataframe_8_12_desc)


########### Models CNN_8_12_8 ###########
(2, 24)
Pire résultat sur le train :  0.8819
Pire Résultat sur la val :  0.5676
Meilleur résultat sur le train :  0.97088
Meilleur résultat sur la val :  0.6258
Moyenne des résultats sur le train :  0.92639
Moyenne des résultats sur la val :  0.5967
Mediane des résultat sur le train :  0.92639
Mediane des résultat sur la val :  0.5967

########### Models CNN_8_12_10 ###########
(2, 24)
Pire résultat sur le train :  0.95494
Pire Résultat sur la val :  0.5669
Meilleur résultat sur le train :  0.96242
Meilleur résultat sur la val :  0.6474
Moyenne des résultats sur le train :  0.95868
Moyenne des résultats sur la val :  0.60715
Mediane des résultat sur le train :  0.95868
Mediane des résultat sur la val :  0.60715


A premier vu les résultats semblent pertinent , étant donnée que les modèles n'overfitent pas
et ont une bonne val accuracy,
<h2>les meilleurs modèles 

In [140]:
filtered_8_12=print_models_filtered(dataframe_8_12,dataframe_8_12_desc)


########### Models CNN_8_12_8 ###########
(1, 24)
Pire résultat sur le train :  0.97088
Pire Résultat sur la val :  0.6258
Meilleur résultat sur le train :  0.97088
Meilleur résultat sur la val :  0.6258
Moyenne des résultats sur le train :  0.97088
Moyenne des résultats sur la val :  0.6258
Mediane des résultat sur le train :  0.97088
Mediane des résultat sur la val :  0.6258

########### Models CNN_8_12_10 ###########
(1, 24)
Pire résultat sur le train :  0.96242
Pire Résultat sur la val :  0.6474
Meilleur résultat sur le train :  0.96242
Meilleur résultat sur la val :  0.6474
Moyenne des résultats sur le train :  0.96242
Moyenne des résultats sur la val :  0.6474
Mediane des résultat sur le train :  0.96242
Mediane des résultat sur la val :  0.6474


les meilleurs modèles nous confirme ça, 
essayons un peu de régularisation:
<h5> Test avec Régularisation

In [79]:
print_models(dataframe_8_12_reg,dataframe_8_12_reg_desc)


########### Models CNN_8_12_R_8 ###########
(4, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.36302
Meilleur résultat sur la val :  0.3629
Moyenne des résultats sur le train :  0.16575499999999999
Moyenne des résultats sur la val :  0.16572499999999998
Mediane des résultat sur le train :  0.1
Mediane des résultat sur la val :  0.1


<h5>on remarque ici que les modèles ne sont pas du tout pertinents, c'est le fait de lancer plusieurs modèles avec un random dans l'espoir de trouver un modèle mieux que les précedents,
ma résuglarisation était trop forte , j'ai donc baissé pour voir si ça ira mieux, et j'ai également ajouté du dropout:
<h5> Test avec Régularisation + Dropout:

In [141]:
filtered_8_12_reg=print_models_filtered(dataframe_8_12_reg,dataframe_8_12_reg_desc)


########### Models CNN_8_12_R_8 ###########
(1, 24)
Pire résultat sur le train :  0.36302
Pire Résultat sur la val :  0.3629
Meilleur résultat sur le train :  0.36302
Meilleur résultat sur la val :  0.3629
Moyenne des résultats sur le train :  0.36302
Moyenne des résultats sur la val :  0.3629
Mediane des résultat sur le train :  0.36302
Mediane des résultat sur la val :  0.3629


on remarque ici 
<h2> Test avec Régularisation +Dropout:

In [180]:
print_models(dataframe_8_12_reg_drop,dataframe_8_12_reg_drop_desc)


########### Models CNN_8_12_R_D_8 ###########
(27, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.9919600000000001
Meilleur résultat sur la val :  0.7313
Moyenne des résultats sur le train :  0.7075466666666669
Moyenne des résultats sur la val :  0.5433444444444445
Mediane des résultat sur le train :  0.86144
Mediane des résultat sur la val :  0.6567

########### Models CNN_8_12_R_D_10 ###########
(8, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.8558
Meilleur résultat sur la val :  0.7152
Moyenne des résultats sur le train :  0.7046975
Moyenne des résultats sur la val :  0.6133875
Mediane des résultat sur le train :  0.77936
Mediane des résultat sur la val :  0.6844


on remarque ici :
<h2>les meilleurs modèles:

In [186]:
filtered_8_12_reg_drop=print_models_filtered(dataframe_8_12_reg_drop,dataframe_8_12_reg_drop_desc)


########### Models CNN_8_12_R_D_8 ###########
(12, 24)
Pire résultat sur le train :  0.7261
Pire Résultat sur la val :  0.6571
Meilleur résultat sur le train :  0.9919600000000001
Meilleur résultat sur la val :  0.7313
Moyenne des résultats sur le train :  0.8975983333333333
Moyenne des résultats sur la val :  0.6923750000000001
Mediane des résultat sur le train :  0.94513
Mediane des résultat sur la val :  0.6874

########### Models CNN_8_12_R_D_10 ###########
(4, 24)
Pire résultat sur le train :  0.7744
Pire Résultat sur la val :  0.6845
Meilleur résultat sur le train :  0.8558
Meilleur résultat sur la val :  0.7152
Moyenne des résultats sur le train :  0.80446
Moyenne des résultats sur la val :  0.6933499999999999
Mediane des résultat sur le train :  0.79382
Mediane des résultat sur la val :  0.68685


donc
<h2> On récapitule sur les meilleurs modèles avec  8 à 12 couches 

In [178]:
best_models_print([filtered_8_12,filtered_8_12_reg,filtered_8_12_reg_drop],8,12).sort_values('val_accuracy',ascending=False)


########### Models best_CNN_8_12 ###########
(19, 24)
Pire résultat sur le train :  0.36302
Pire Résultat sur la val :  0.3629
Meilleur résultat sur le train :  0.9919600000000001
Meilleur résultat sur la val :  0.7313
Moyenne des résultats sur le train :  0.8571231578947367
Moyenne des résultats sur la val :  0.6693684210526315
Mediane des résultat sur le train :  0.8793
Mediane des résultat sur la val :  0.6845


,Conv2D_nb,Conv2D_size,Conv2D_activation,output_activation,Conv2D_padding,MaxPooling2D_use,MaxPooling2D_values,MaxPooling2D_Position,use_dropout,dropout_indexes,...,l2_value,regul_kernel_indexes,loss,optimizer,metrics,epochs,model_id,train_accuracy,val_accuracy,dossier
3,8,"(128, 4) (128, 4) (64, 4) (64, 4) (64, 4) (32,...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,7,sparse_categorical_crossentropy,str,accuracy,50,1760125,0.98344,0.7313,20200127
5,8,"(128, 4) (128, 4) (64, 4) (64, 4) (64, 4) (32,...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,7,sparse_categorical_crossentropy,str,accuracy,30,275590,0.95916,0.7291,20200128
4,8,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,6,sparse_categorical_crossentropy,str,accuracy,150,6817867,0.99196,0.7287,20200127
1,8,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,6,sparse_categorical_crossentropy,str,accuracy,50,1799197,0.97602,0.7199,20200127
0,8,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,NaN,sparse_categorical_crossentropy,str,accuracy,30,6122335,0.97594,0.7174,20200127
4,10,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 5,...,0.005,3 8,sparse_categorical_crossentropy,str,accuracy,30,8435598,0.85580,0.7152,20200127
2,8,"(64, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64,...",relu,softmax,same,True,4,1 4,True,0 4 6,...,0.025,6,sparse_categorical_crossentropy,str,accuracy,50,5084237,0.95958,0.7042,20200127
0,10,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0,...,0.005,3 8,sparse_categorical_crossentropy,str,accuracy,30,554403,0.80332,0.6870,20200127
3,10,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 5,...,0.005,3,sparse_categorical_crossentropy,str,accuracy,30,9843126,0.78432,0.6867,20200127
5,10,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 4,True,0 5,...,0.005,3 8,sparse_categorical_crossentropy,str,accuracy,30,2886425,0.77440,0.6845,20200127


<h2> Résultats du modèle 12 à 15 couches sans régularisation:

In [168]:
print_models(dataframe_12_15,dataframe_12_15_desc)


########### Models CNN_12_15_12 ###########
(1, 24)
Pire résultat sur le train :  0.8880600000000001
Pire Résultat sur la val :  0.6032
Meilleur résultat sur le train :  0.8880600000000001
Meilleur résultat sur la val :  0.6032
Moyenne des résultats sur le train :  0.8880600000000001
Moyenne des résultats sur la val :  0.6032
Mediane des résultat sur le train :  0.8880600000000001
Mediane des résultat sur la val :  0.6032


In [174]:
filtered_12_15=print_models_filtered(dataframe_12_15,dataframe_12_15_desc)

<h5> On remarque  ici que les modèles importés à 12 couches sont en moyenne de 0.60 en val contre 0.88 ( overfiting)
    on va assayer une régularisation l1_l2
<h5> Test avec Régularisation

In [158]:
print_models(dataframe_12_15_reg,dataframe_12_15_reg_desc)


########### Models CNN_12_15_R_12 ###########
(3, 24)
Pire résultat sur le train :  0.67648
Pire Résultat sur la val :  0.5968
Meilleur résultat sur le train :  0.71846
Meilleur résultat sur la val :  0.6499
Moyenne des résultats sur le train :  0.6922
Moyenne des résultats sur la val :  0.6205666666666667
Mediane des résultat sur le train :  0.68166
Mediane des résultat sur la val :  0.615


On remarque ici : que les améliorations apportées à l'overfiting sont remarquables, cependant
l'accuracy à baissé, 


##### les meilleurs modèles sont:

In [159]:
filtered_12_15_reg=print_models_filtered(dataframe_12_15_reg,dataframe_12_15_reg_desc)


########### Models CNN_12_15_R_12 ###########
(1, 24)
Pire résultat sur le train :  0.71846
Pire Résultat sur la val :  0.6499
Meilleur résultat sur le train :  0.71846
Meilleur résultat sur la val :  0.6499
Moyenne des résultats sur le train :  0.71846
Moyenne des résultats sur la val :  0.6499
Mediane des résultat sur le train :  0.71846
Mediane des résultat sur la val :  0.6499


<h5>d'où on baisse des L1 et L2 et on ajoute  du dropout pour tenter de mieux régulariser et avoir un bon résultat sur la val:
<h5> Test avec Régularisation + Dropout:

In [160]:
print_models(dataframe_12_15_reg_drop, dataframe_12_15_reg_drop_desc)


########### Models CNN_12_15_R_D_12 ###########
(16, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.97282
Meilleur résultat sur la val :  0.7029
Moyenne des résultats sur le train :  0.5462612499999999
Moyenne des résultats sur la val :  0.43891875
Mediane des résultat sur le train :  0.68413
Mediane des résultat sur la val :  0.61285

########### Models CNN_12_15_R_D_15 ###########
(7, 24)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.79732
Meilleur résultat sur la val :  0.7019
Moyenne des résultats sur le train :  0.6255799999999999
Moyenne des résultats sur la val :  0.5689142857142857
Mediane des résultat sur le train :  0.73146
Mediane des résultat sur la val :  0.6653


##### on remarque ici que la moyenne  et la médiane sont proches, mais la val accuracy est toujours basse par rapport aux modèles précedents , malgré plusieurs tentatives d'utilisation des dropout, regularisation, augmentation du batch size et des epochs
## les meilleurs modèles sont:


In [187]:
filtered_12_15_reg_drop=print_models_filtered(dataframe_12_15_reg_drop, dataframe_12_15_reg_drop_desc)


########### Models CNN_12_15_R_D_12 ###########
(8, 24)
Pire résultat sur le train :  0.763
Pire Résultat sur la val :  0.6444
Meilleur résultat sur le train :  0.97282
Meilleur résultat sur la val :  0.7029
Moyenne des résultats sur le train :  0.8769325
Moyenne des résultats sur la val :  0.6674125
Mediane des résultat sur le train :  0.88579
Mediane des résultat sur la val :  0.6606

########### Models CNN_12_15_R_D_15 ###########
(3, 24)
Pire résultat sur le train :  0.73146
Pire Résultat sur la val :  0.667
Meilleur résultat sur le train :  0.79732
Meilleur résultat sur la val :  0.7019
Moyenne des résultats sur le train :  0.7547
Moyenne des résultats sur la val :  0.6818666666666666
Mediane des résultat sur le train :  0.73532
Mediane des résultat sur la val :  0.6767


##### au final essayons de récapituler sur l'ensemble des modèles du 12 à 15 de conv

In [175]:
best_models_print([filtered_12_15_reg_rep,filtered_12_15_reg,filtered_12_15],12,15).sort_values('val_accuracy',ascending=False)


########### Models best_CNN_12_15 ###########
(12, 24)
Pire résultat sur le train :  0.71846
Pire Résultat sur la val :  0.6444
Meilleur résultat sur le train :  0.97282
Meilleur résultat sur la val :  0.7029
Moyenne des résultats sur le train :  0.8331683333333334
Moyenne des résultats sur la val :  0.6695666666666668
Mediane des résultat sur le train :  0.79867
Mediane des résultat sur la val :  0.6669


,Conv2D_nb,Conv2D_size,Conv2D_activation,output_activation,Conv2D_padding,MaxPooling2D_use,MaxPooling2D_values,MaxPooling2D_Position,use_dropout,dropout_indexes,...,l2_value,regul_kernel_indexes,loss,optimizer,metrics,epochs,model_id,train_accuracy,val_accuracy,dossier
14,12,"(64, 2) (64, 2) (128, 2) (128, 2) (226, 2) (22...",relu,softmax,same,True,2,1 4 7 11,True,0 5,...,0.002,3 8,sparse_categorical_crossentropy,str,accuracy,30,7721977,0.91938,0.7029,20200127
0,15,"(64, 2) (64, 2) (64, 2) (64, 2) (64, 2) (128, ...",selu,softmax,same,True,2,2 4 8 12,True,0 6 14,...,0.095,10 12,sparse_categorical_crossentropy,str,accuracy,40,4724522,0.79732,0.7019,20200204
15,12,"(64, 2) (64, 2) (128, 2) (128, 2) (226, 2) (22...",relu,softmax,same,True,2,1 4 7 11,True,0 5,...,0.005,3 8,sparse_categorical_crossentropy,str,accuracy,50,3674271,0.95960,0.6923,20200127
4,15,"(64, 2) (64, 2) (64, 2) (64, 2) (64, 2) (128, ...",selu,softmax,same,True,2,2 4 8 12,True,0 6 14,...,0.099,10 12,sparse_categorical_crossentropy,str,accuracy,40,6824931,0.73532,0.6767,20200205
5,12,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 6,True,0 6,...,0.037,4 9 11,sparse_categorical_crossentropy,str,accuracy,100,8720999,0.96632,0.6720,20200128
3,15,"(64, 2) (64, 2) (64, 2) (64, 2) (64, 2) (128, ...",selu,softmax,same,True,2,2 4 8 12,True,0 6 14,...,0.095,10 12,sparse_categorical_crossentropy,str,accuracy,40,4348823,0.73146,0.6670,20200205
8,12,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 6,True,0 6,...,0.037,4 9 11,sparse_categorical_crossentropy,str,accuracy,40,417064,0.85220,0.6668,20200130
3,12,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 6,True,0 6,...,0.037,4 9 11,sparse_categorical_crossentropy,str,accuracy,30,3539315,0.80002,0.6544,20200128
4,12,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 6,True,0 6,...,0.037,4 9 11,sparse_categorical_crossentropy,str,accuracy,30,1973777,0.78212,0.6543,20200128
7,12,"(128, 4) (128, 4) (128, 4) (64, 4) (64, 4) (64...",relu,softmax,same,True,4,1 6,True,0 6,...,0.037,4 9 11,sparse_categorical_crossentropy,str,accuracy,30,9397853,0.76300,0.6522,20200130


## Résultats finaux:

![best_12_15](./images_from_tensorBoard/best_12_12.png)

#### les meilleurs résultats tirés de ce type de modèles 

In [204]:
best_cnn_find=[]
best_cnn_find.append(best_models_print([filtered_12_15_reg_drop,filtered_12_15_reg,filtered_12_15],12,15).sort_values('val_accuracy',ascending=False))
best_cnn_find.append(best_models_print([filtered_8_12_reg_drop,filtered_8_12_reg,filtered_8_12],8,12).sort_values('val_accuracy',ascending=False))
best_cnn_find.append(best_models_print([filtered_6_8_reg_drop,filtered_6_8_reg,filtered_6_8],6,8).sort_values('val_accuracy',ascending=False))
best_cnn_find.append(best_models_print([filtered_3_6_reg_drop,filtered_3_6_reg,filtered_3_6],3,6).sort_values('val_accuracy',ascending=False))
best_cnn_find.append(best_models_print([filtered_3_3_reg_drop,filtered_3_3_reg,filtered_3_3],3,3).sort_values('val_accuracy',ascending=False))


########### Models best_CNN_12_15 ###########
(12, 24)
Pire résultat sur le train :  0.71846
Pire Résultat sur la val :  0.6444
Meilleur résultat sur le train :  0.97282
Meilleur résultat sur la val :  0.7029
Moyenne des résultats sur le train :  0.8331683333333334
Moyenne des résultats sur la val :  0.6695666666666668
Mediane des résultat sur le train :  0.79867
Mediane des résultat sur la val :  0.6669

########### Models best_CNN_8_12 ###########
(19, 24)
Pire résultat sur le train :  0.36302
Pire Résultat sur la val :  0.3629
Meilleur résultat sur le train :  0.9919600000000001
Meilleur résultat sur la val :  0.7313
Moyenne des résultats sur le train :  0.8571231578947369
Moyenne des résultats sur la val :  0.6693684210526316
Mediane des résultat sur le train :  0.8793
Mediane des résultat sur la val :  0.6845

########### Models best_CNN_6_8 ###########
(28, 24)
Pire résultat sur le train :  0.36302
Pire Résultat sur la val :  0.3629
Meilleur résultat sur le train :  0.9919600000

In [208]:
best_models_print([best_cnn_find],3,15).sort_values('val_accuracy',ascending=False)


########### Models best_CNN_3_15 ###########
(148, 24)
Pire résultat sur le train :  0.36302
Pire Résultat sur la val :  0.3629
Meilleur résultat sur le train :  0.9995200000000001
Meilleur résultat sur la val :  0.7603
Moyenne des résultats sur le train :  0.8167605405405406
Moyenne des résultats sur la val :  0.6947033783783784
Mediane des résultat sur le train :  0.813
Mediane des résultat sur la val :  0.7199


,Conv2D_nb,Conv2D_size,Conv2D_activation,output_activation,Conv2D_padding,MaxPooling2D_use,MaxPooling2D_values,MaxPooling2D_Position,use_dropout,dropout_indexes,...,l2_value,regul_kernel_indexes,loss,optimizer,metrics,epochs,model_id,train_accuracy,val_accuracy,dossier
21,5,"(64, 3) (128, 3) (64, 3) (32, 3) (32, 3)",selu,softmax,same,True,3,2 4,False,0,...,0.060,1 3,sparse_categorical_crossentropy,str,accuracy,35,871806,0.83308,0.7603,20200203
9,6,"(64, 3) (128, 3) (128, 3) (64, 3) (32, 3) (32, 3)",relu,softmax,same,True,3,4 5,False,0,...,0.040,1 4,sparse_categorical_crossentropy,str,accuracy,25,4029029,0.84328,0.7547,20200201
17,6,"(64, 3) (128, 3) (128, 3) (64, 3) (32, 3) (32, 3)",relu,softmax,same,True,3,4 5,False,0,...,0.040,1 4,sparse_categorical_crossentropy,str,accuracy,40,5524285,0.89076,0.7520,20200201
30,5,"(64, 3) (128, 3) (64, 3) (32, 3) (32, 3)",selu,softmax,same,True,3,2 4,False,0,...,0.075,1 3,sparse_categorical_crossentropy,str,accuracy,45,500504,0.82690,0.7514,20200203
16,5,"(64, 3) (128, 3) (64, 3) (32, 3) (32, 3)",selu,softmax,same,True,3,2 4,False,0,...,0.054,1 3,sparse_categorical_crossentropy,str,accuracy,35,640730,0.81538,0.7503,20200203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2,"(64, 5) (64, 5)",selu,softmax,same,True,5,1,False,NaN,...,0.093,2 2,sparse_categorical_crossentropy,str,['accuracy'],30,4602469,0.57868,0.5707,20200126
3,1,"(64, 2)",softmax,softmax,same,False,2,NaN,False,NaN,...,0.000,NaN,sparse_categorical_crossentropy,str,['accuracy'],30,6981395,0.99952,0.5585,20200126
10,3,"(64, 2) (64, 2) (256, 2)",softplus,softmax,same,True,2,2 3,False,NaN,...,0.094,3 1 2,sparse_categorical_crossentropy,str,['accuracy'],30,5161805,0.63336,0.5543,20200126
0,8,"(64, 3) (32, 3) (64, 3) (32, 3) (32, 3) (64, 3...",selu,softmax,same,False,3,NaN,False,NaN,...,0.030,8 2 8 6 5 8 5,sparse_categorical_crossentropy,str,['accuracy'],30,3046826,0.36302,0.3629,20200126


# Conclusion sur les CNN

### le meilleur modèle est donc  avec 0.76% de val accuracy c'est un modèle formé de 5 couches de Conv2D,avec seulement de la régularisation L1=0.15,L2= et sur les couches 1 et 3  , et 2 couches de MaxPooling de kernel 3 , en position 2 et 4 ,

-ces tests nous confirment que les modèles simples et moyens sont pertinents, on pourra eventuellement les améliorer avec plus de régularisation.